In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import operator
import scipy.spatial.distance as distance
import sklearn.metrics

In [2]:
df = pd.read_pickle('processed_df.pkl')
ratings = pd.read_pickle('ratings_sample.pkl')
ratings = ratings.reset_index()

In [3]:
movies = df.drop(columns = ['movieId', 'title_eng', 'year'])

## Build Recommendations for 1 User
Read notes again and make sure this method is in line with what I was initially thinking           
To Do:
- Extend to all users (function)
- Test/train split so hold out some recommendations to evaluate against --> evaluate

Extensions:
- Include year (decade)?
- Include text from description or genome tags
- Downweight older ratings 

In [28]:
# limit to one user
ratings_user = ratings[ratings.userId == 33]
movies_user = df[df.movieId.isin(ratings_user.movieId.unique())]

# keep track of movies already watched
watched = movies_user.movieId.unique()

movies_user = movies_user.drop(columns = ['movieId', 'title_eng', 'year'])

In [5]:
# normalize ratings: subtract mean rating  from ratings
# if rating < mean, normalized rating will be negative. Which is worse than 0 aka not rating movie at all.

#### should this be averaged??

mean_rating = np.mean(ratings_user.rating)
ratings_user.rating = ratings_user.rating - mean_rating

c:\users\gimli\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [6]:
# create user profile: multiply item profile by user ratings --> sum of ratings for each attribute 
profile = movies_user.T.dot(ratings_user.rating.values)

In [17]:
# cosine similarity between movie and user profile 
movies = df.drop(columns = ['movieId', 'title_eng', 'year'])
recommendations = sklearn.metrics.pairwise.cosine_similarity(movies, np.asmatrix(profile.values))

In [23]:
recommendations = pd.DataFrame(recommendations)
recommendations.columns = ['prediction']

# merge to get title
recommendations = pd.merge(recommendations, df[['title_eng', 'movieId']], left_index = True, right_index = True, how = 'left')

# sort 
recommendations = recommendations.sort_values('prediction', ascending = False)

In [30]:
# remove recommendations already watched
recommendations = recommendations[~recommendations.movieId.isin(watched)]

In [31]:
recommendations

,prediction,title_eng,movieId
9372,0.337223,Batman Begins (2005),33794
34855,0.323734,The Mediator (1990),169978
44751,0.323734,Collider (2018),207197
38786,0.323734,Battle Planet (2008),184053
42293,0.323734,Molly (2017),196703
...,...,...,...
4531,-0.375370,Novocaine (2001),4898
20129,-0.375370,Docks of New Orleans (1948),117608
14197,-0.381524,Karthik Calling Karthik (2010),82097
22958,-0.381524,Inseparable (2011),128273
